Paul Hollywood bakes two types of cakes: cheesecakes and black forest cakes.
During any month, he can bake at most 65 cakes in total. The costs per cake and the demands for cakes,
which must be met in time, are given in the following table.

|             | Month 1|     Month 1 | Month 2|    Month 2  | Month 3|    Month 3  |
|:-----------:|:------:|:-----------:|:------:|:-----------:|:------:|:-----------:|
| Item        | Demand | Cost/cake($)| Demand | Cost/cake($)| Demand | Cost/cake($)|
| Cheesecake  | 40     |   3.00      | 30     |   3.40      | 20     |   3.80      |
| Black Forest| 20     |   2.50      | 30     |   2.80      | 10     |   3.40      |

We assume that cakes baked during a month can be used to meet demand for this month. At the end of
each month (after all cakes have been baked and the current month’s demand has been satisfied), a holding
cost of 50 cents per cheesecake and 40 cents per black forest cake is incurred for cakes left in inventory.
Those cakes can be used to satisfy future demand.

In [1]:
using JuMP, HiGHS

# defining model
cake_model = Model(HiGHS.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: HiGHS

In [2]:
# defining variables to index over
cakes = [:cheesecake, :blackforest]
months = 1:3

1:3

In [3]:
# dictionary to store the holding costs 
holding = [0.50, 0.40]
holding_costs = Dict(zip(cakes, holding))

using NamedArrays

# NamedArray for demands and production costs 
demand_mat = [ 40 30 20; 20 30 10 ]
demands = NamedArray( demand_mat, (cakes, months), ("cake", "month") )

prod_mat = [3.00 3.40 3.80; 2.50 2.80 3.40]
production_costs = NamedArray(prod_mat, (cakes, months), ("cake", "month"))

# variables for number of cakes to make each month
@variable(cake_model, x[cakes, months] >= 0)

# variable to represent the numver of cakes left in inventory at the end of each month
@variable(cake_model, h[cakes, months] >= 0)

2-dimensional DenseAxisArray{VariableRef,2,...} with index sets:
    Dimension 1, [:cheesecake, :blackforest]
    Dimension 2, 1:3
And data, a 2×3 Matrix{VariableRef}:
 h[cheesecake,1]   h[cheesecake,2]   h[cheesecake,3]
 h[blackforest,1]  h[blackforest,2]  h[blackforest,3]

In [4]:
# defining constraints
@constraint(cake_model, total_cakes[i in months], sum(x[j, i] for j in cakes) <= 65)

@constraint(cake_model, monthly_demand1, x[:cheesecake, 1] >= demands[1, 1])
@constraint(cake_model, monthly_demand2, x[:blackforest, 1] >= demands[2, 1])
@constraint(cake_model, monthly_demand3, sum(h[:cheesecake, 1] + x[:cheesecake, 2]) >= demands[1, 2])
@constraint(cake_model, monthly_demand4, sum(h[:blackforest, 1] + x[:blackforest, 2]) >= demands[2, 2])
@constraint(cake_model, monthly_demand5, sum(h[:cheesecake, 2] + x[:cheesecake, 3]) >= demands[1, 3])
@constraint(cake_model, monthly_demand6, sum(h[:blackforest, 2] + x[:blackforest, 3]) >= demands[2, 3])


monthly_demand6 : x[blackforest,3] + h[blackforest,2] ≥ 10.0

In [5]:
# objective function 
prod_cost = sum(x[i,j]*production_costs[i,j] for i in cakes for j in months)
hold_cost = sum(h[i,j]*holding_costs[i] for i in cakes for j in months)

@objective(cake_model, Min, prod_cost + hold_cost)

print(cake_model)

In [6]:
optimize!(cake_model)

Presolving model
7 rows, 8 cols, 12 nonzeros
0 rows, 0 cols, 0 nonzeros
Presolve : Reductions: rows 0(-9); columns 0(-12); elements 0(-16) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  2.1100000000e+02
HiGHS run time      :          0.00
